In [1]:
import pandas as pd
import numpy as np
import sys
import ast
import collections
import networkx as nx
from pyvis.network import Network
import plotly.graph_objects as go
sys.path.append('../')
from modules import utils, constants
import matplotlib.pyplot as plt
%matplotlib inline

#### The data

In [2]:
training_df = pd.read_csv('../data/missingness/0/training_set.csv')
training_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,1,0,0,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,1,1,1
4,1,0,0,1,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,1,1


In [3]:
testing_df = pd.read_csv('../data/missingness/0/testing_set.csv')
testing_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
2,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,1,0,0,0,1
3,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,1
4,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1


In [4]:
model_test_df = pd.read_csv('../test_dfs/negative_reward/step_30_correct_1_incorrect_-1.csv')#.drop(['index'], axis=1)
model_test_df.head()

,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success
0,0.0,17.0,0.515152,1.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,1.0
1,1.0,11.0,0.696970,1.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,1.0
2,2.0,16.0,0.545455,1.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,1.0
3,3.0,15.0,0.575758,1.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,1.0
4,4.0,9.0,0.757576,1.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,1.0


In [5]:
merged_test_df = pd.concat([testing_df.reset_index(drop=True),model_test_df.reset_index(drop=True)], axis=1)
merged_test_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,anti_smith_antibody,label,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0.0,17.0,0.515152,1.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,1.0
1,1,1,0,0,0,0,0,0,0,0,...,0,1,1.0,11.0,0.696970,1.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,1.0
2,1,0,0,0,0,0,1,0,0,0,...,0,1,2.0,16.0,0.545455,1.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,1.0
3,1,1,0,0,0,0,0,0,1,0,...,0,1,3.0,15.0,0.575758,1.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,1.0
4,1,0,0,0,0,0,0,0,1,0,...,0,1,4.0,9.0,0.757576,1.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,1.0


#### Adding a score to the datasets

In [6]:
# Dictionary with the weignts of the different features
criteria_weights = {'fever':2, 'leukopenia':3, 'thrombocytopenia':4, 'auto_immune_hemolysis':4, 'delirium':2, 'psychosis':3, 'seizure':5, 
                    'non_scarring_alopecia':2, 'oral_ulcers':2, 'subacute_cutaneous_lupus':4, 'discoid_lupus':4, 'acute_cutaneous_lupus':6, 
                    'pleural_effusion': 5, 'pericardial_effusion':5, 'acute_pericarditis':6, 'joint_involvement':6, 'proteinuria':4, 
                    'anti_cardioliphin_antibodies':2, 'anti_β2gp1_antibodies':2, 'lupus_anti_coagulant':2, 'low_c3':3, 'low_c4':3, 
                    'low_c3_and_low_c4':4, 'low_c3_or_low_c4':3, 'anti_dsdna_antibody':6, 'anti_smith_antibody':6}

# Dictionary mapping features to their respective domains
domains_feat_dict = {'constitutional': ['fever'],
                     'hematologic': ['leukopenia', 'thrombocytopenia', 'auto_immune_hemolysis'],
                     'neuropsychiatric': ['delirium', 'psychosis', 'seizure'],
                     'mucocutaneous': ['non_scarring_alopecia', 'oral_ulcers', 'cutaneous_lupus'],
                     'serosal': ['pleural_effusion', 'pericardial_effusion', 'acute_pericarditis'],
                     'musculoskeletal': ['joint_involvement'],
                     'renal': ['proteinuria'],
                     'antiphospholipid_antibodies': ['anti_cardioliphin_antibodies', 'anti_β2gp1_antibodies', 'lupus_anti_coagulant'],
                     'complement_proteins': ['low_c3', 'low_c4'],
                     'sle_specific_antibodies':['anti_dsdna_antibody', 'anti_smith_antibody']}

# Dicionary with the maximum possible score for each domain
domains_max_scores_dict = {'constitutional': 2, 'hematologic': 4, 'neuropsychiatric': 5, 'mucocutaneous': 6, 'serosal': 6, 'musculoskeletal': 6, 
                           'renal': 4, 'antiphospholipid_antibodies': 2, 'complement_proteins': 4, 'sle_specific_antibodies':6}



def get_cutaneous_lupus_score(cutaneous_type):
    '''
    Computes the score given a value for the cutaneous lupus feature
    '''
    if cutaneous_type == 0: #negative for any form of cutaneous lupus
        return 0
    elif cutaneous_type == 1: #subacute cutaneous lupus
        return criteria_weights['subacute_cutaneous_lupus']
    elif cutaneous_type == 2: #acute cutaneous lupus
        return criteria_weights['acute_cutaneous_lupus']
    elif cutaneous_type == 3: #discoid lupus
        return criteria_weights['discoid_lupus']
    else:
        raise Exception('Unknown cutaneous lupus type!')

def get_feat_score(row, feat):
    '''
    Computes the score for a given feature in a row
    '''
    if feat == 'cutaneous_lupus':
        feat_score = get_cutaneous_lupus_score(row[feat])
    elif row[feat] <= 0:
        feat_score = 0
    else:
        feat_score = criteria_weights[feat]
    return feat_score


def get_c3_c4_score(c3, c4): 
    '''
    Computes the score for the complements proteins domain
    '''
    if (c3 == 1) & (c4 == 1): # 1 is low, 0 is not low
        return criteria_weights['low_c3_and_low_c4']
    elif (c3 == 1) | (c4 == 1):
        return criteria_weights['low_c3_or_low_c4']
    else:
        return 0

def get_domain_score(row, domain):
    '''
    Computes the score for a given domain in a specified row
    '''
    domain_features = domains_feat_dict[domain] # a list of features in that domain
    domain_score = 0
    if domain == 'complement_proteins':
        domain_score = get_c3_c4_score(row.low_c3, row.low_c4)
    else:
        for feat in domain_features:
            feat_score = get_feat_score(row, feat)
            if feat_score > domain_score:
                domain_score = feat_score
    if domain_score > domains_max_scores_dict[domain]:
        raise Exception('The score is too large for this domain!')
    return domain_score

def compute_score(row):
    '''
    Computes the total score of a row in a dataframe
    '''
    if row['ana'] == 0: # 0=negative, 1=positive 
        return 0
    total_row_score = 0
    for domain in domains_feat_dict.keys():
        domain_score = get_domain_score(row, domain)
        total_row_score += domain_score
    return total_row_score

In [12]:
merged_test_df['score'] = merged_test_df.apply(lambda row: compute_score(row), axis=1)
merged_test_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,label,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success,score
0,1,0,0,0,0,0,0,0,0,0,...,1,0.0,17.0,0.515152,1.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,1.0,17
1,1,1,0,0,0,0,0,0,0,0,...,1,1.0,11.0,0.696970,1.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,1.0,12
2,1,0,0,0,0,0,1,0,0,0,...,1,2.0,16.0,0.545455,1.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,1.0,19
3,1,1,0,0,0,0,0,0,1,0,...,1,3.0,15.0,0.575758,1.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,1.0,12
4,1,0,0,0,0,0,0,0,1,0,...,1,4.0,9.0,0.757576,1.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,1.0,16


#### Functions for drawing pathways

In [7]:
def generate_tuple_dict(df):
    frequency_dict = {}
    for traj in df.trajectory:
        if traj in frequency_dict.keys():
            frequency_dict[traj] += 1
        else:
            frequency_dict[traj] = 1
    overall_tup_dict = {}
    for key, value in frequency_dict.items():
        new_key = ast.literal_eval(key)
        for tup in zip(new_key, new_key[1:]):
            if tup in overall_tup_dict.keys():
                overall_tup_dict[tup] += value
            else:
                overall_tup_dict[tup] = value
    return overall_tup_dict

In [8]:
def create_sankey_df(df):
    overall_tup_dict = generate_tuple_dict(df)
    sankey_df = pd.DataFrame()
    sankey_df['source'] = [i[0] for i in overall_tup_dict.keys()]
    sankey_df['target'] = [i[1] for i in overall_tup_dict.keys()]
    sankey_df['value'] = list(overall_tup_dict.values())
    sankey_df['link_type'] = sankey_df['target'].apply(lambda i: 'terminal' if i in ['No lupus', 'Lupus', 'Inconclusive diagnosis'] else 'non_terminal')
    return sankey_df

In [9]:
def draw_pyvis_network(pathways_df, pathway_type='network'): #second type is trajectory 
    pathways_df['type'] = 'directed'
    start_node = pathways_df.iloc[0]['source']
    all_nodes = list(set(pathways_df.source.unique().tolist() + pathways_df.target.unique().tolist()))
    terminal_nodes = list(set(pathways_df[pathways_df.link_type=='terminal'].target))
    non_terminal_nodes = [i for i in all_nodes if i not in terminal_nodes and i!=start_node]
    
    #change get_net to something else since it satnds for Game of Thrones
    got_net = Network(notebook=True, height='600px', width='100%', directed=True, cdn_resources='in_line')
    got_net.add_node(start_node, color='purple', size=20)
    got_net.add_nodes(non_terminal_nodes, size=[15]*len(non_terminal_nodes), color=['blue']*len(non_terminal_nodes))
    got_net.add_nodes(terminal_nodes, color=['green']*len(terminal_nodes), size=[20]*len(terminal_nodes))
    for src, target, value, edge_thresh in zip(pathways_df.source, pathways_df.target, pathways_df.value, pathways_df.edge_threshold):
        if pathway_type =='network':
            if value > threshold:
                if math.isnan(edge_thresh):
                    got_net.add_edge(src, target, value=value, color='red')
                else:
                    got_net.add_edge(src, target, value=value, color='red', label=str(int(edge_thresh)))
            else:
                got_net.add_edge(src, target, value=value, color='blue')
        elif pathway_type =='trajectory': #It's just one trajectory
            if isinstance(edge_thresh, str):
                got_net.add_edge(src, target, value=value, color='blue', label=edge_thresh)
            else:
                got_net.add_edge(src, target, value=value, color='blue', label=str(int(edge_thresh)))
        else:
            print('Unknown pathway type')
    return got_net

In [10]:
def get_threshold_value(df, source): #source e.g. 'ana'
    threshold_values = df[source].unique().tolist()
    if len(threshold_values) == 1:
        return int(threshold_values[0])
    else:
        return str(threshold_values)

In [11]:
def draw_one_trajectory(pred_df, trajectory): #pred_df = pred_no_lupus par exemple
    trajectory_df = pred_df[pred_df.trajectory == trajectory]
    trajectory_testing_df = testing_df.loc[trajectory_df.index]
    trajectory_pathways_df = create_sankey_df(trajectory_df)
    trajectory_pathways_df['edge_threshold'] = [get_threshold_value(trajectory_testing_df, source) for source in trajectory_pathways_df.source]
    trajectory_net = draw_pyvis_network(trajectory_pathways_df, 'trajectory')   
    return trajectory_net

#### Misdiagnosed episodes - Diagnosed as NO LUPUS instead of lupus

In [13]:
misdiagnosed_as_no_lupus = merged_test_df[(merged_test_df.y_pred == 0) & (merged_test_df.y_actual !=0)]
misdiagnosed_as_no_lupus.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,label,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success,score
5,1,0,0,1,0,1,0,0,1,0,...,1,5.0,11.0,-1.303030,0.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,0.0,11
22,1,1,0,0,0,0,0,0,0,0,...,1,22.0,15.0,-1.424242,0.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,0.0,11
175,1,0,0,0,0,0,0,0,1,0,...,1,175.0,20.0,-1.575758,0.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,0.0,10
178,1,1,0,0,0,0,0,0,0,0,...,1,178.0,18.0,-1.515152,0.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,0.0,13
239,1,0,0,1,1,0,0,0,0,0,...,1,239.0,12.0,-1.333333,0.0,1.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,0.0,10


In [14]:
misdiagnosed_as_no_lupus.score.mean(), misdiagnosed_as_no_lupus.score.std()

(11.362292051756008, 1.5805477003451462)

In [15]:
misdiagnosed_as_no_lupus.score.min(), misdiagnosed_as_no_lupus.score.max()

(10, 19)

In [23]:
misdiagnosed_as_no_lupus.episode_length.mean(), misdiagnosed_as_no_lupus.episode_length.std()

(15.67097966728281, 3.639147840008382)

In [20]:
commonest_pathway = misdiagnosed_as_no_lupus.trajectory.value_counts().index[0]

In [21]:
commonest_pathway_net = draw_one_trajectory(misdiagnosed_as_no_lupus, commonest_pathway)
commonest_pathway_net.show('Example.html')

#### Misdiagnosed episodes - Diagnosed as LUPUS instead of No Lupus

In [24]:
misdiagnosed_as_lupus = merged_test_df[(merged_test_df.y_pred == 1) & (merged_test_df.y_actual !=1)]
misdiagnosed_as_lupus.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,label,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success,score
67,1,0,1,0,0,0,0,0,0,0,...,0,67.0,8.0,-1.212121,1.0,0.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,0.0,9
709,1,0,1,0,0,0,0,0,1,0,...,0,709.0,16.0,-1.454545,1.0,0.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,0.0,9
715,1,0,0,0,0,1,0,0,0,0,...,0,715.0,23.0,-1.666667,1.0,0.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,0.0,9
915,1,0,1,0,0,0,0,0,0,0,...,0,915.0,8.0,-1.212121,1.0,0.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,0.0,9
1079,1,0,1,0,0,0,0,0,0,0,...,0,1079.0,16.0,-1.454545,1.0,0.0,"['ana', 'pericardial_effusion', 'leukopenia', ...",0.0,0.0,9


In [25]:
misdiagnosed_as_lupus.score.mean(), misdiagnosed_as_lupus.score.std()

(8.632075471698114, 0.5576390998471566)

In [ ]:
misdiagnosed_as_lupus.score.min(), misdiagnosed_as_lupus.score.max()

In [28]:
commonest_pathway = misdiagnosed_as_lupus.trajectory.value_counts().index[0]

In [30]:
commonest_pathway_net = draw_one_trajectory(misdiagnosed_as_lupus, commonest_pathway)
commonest_pathway_net.show('Example.html')

#### Drawing pathways